# VotaMeglio: analisi dei dati e KPI

*Lena Emanuele, Alexandru Paraschiv Dumitru, Urban Denis, Josè Zito Dalla Torre*

L'obiettivo di questo notebook è definire delle proposte di KPI per **misurare la diffusione del metodo del giudizio maggioritario**. La misurazione di questo obiettivo può essere effettuata tenendo traccia di due fenomeni:

1. l'utilizzo del metodo per dare voti (diffusione dal pt. di vista del votante)
2. la creazione di nuovi sondaggi che fanno uso del metodo maggioritario (diffusione dal pt. di vista del proponente)

Su questi fenomeni si propongono delle metriche, per il momento ricavate a partire da i dati sulle visite e sul numero di utenti della piattaforma. 

## Estrazione dei dati sul numero di utenti

Per estrarre i dati sul numero di utenti e visite della piattaforma, si generano due report di Google Analytic: 

1. il primo riporta le visite e gli utenti sulle pagine specifiche (identificate dal path univoco) 
2. il secondo riporta le visite e gli utenti su intere "classi di pagine" (es. tutte le specifiche pagine di voto con un criterio dei vari sondaggi vengono raggruppate in una voce sola)

In [71]:
import pandas as pd

In [72]:
def parse_ga_report(filename) -> pd.DataFrame:
    """read a GA report and extract 
    the main table as pandas dataframe"""

    # read file content
    with open(filename, 'r') as f:
        lines: list = f.readlines()
        f.close()

    content = ""
    for line in lines: 
        content += line

    # isolate various tables
    headers_and_tables = content.split("\n\n")

    # make a dataframe from the main data table
    table_lines = headers_and_tables[-1].split('\n')
    df = pd.DataFrame([x.split(',') for x in table_lines[1:]])
    
    # set table col names
    df.columns = table_lines[0].split(",")

    return df


In [73]:
users_on_pages = parse_ga_report("data/070123/input/utenti-su-pagina.csv")
users_on_pages_classes = parse_ga_report("data/070123/input/utenti-su-classe-pagina.csv")

In [74]:
users_on_pages.head(5)

,Percorso pagina e classe schermata,Visualizzazioni,Utenti,Visualizzazioni per utente,Durata media del coinvolgimento,Conteggio eventi,Conversioni,Entrate totali
0,/,339,52,6.519230769230769,85.76923076923077,991,0,0
1,/gestione-sondaggi/,59,21,2.8095238095238093,21.61904761904762,158,0,0
2,/voti-e-risultati/1/vota/giudizio-maggioritario/,39,22,1.7727272727272727,32.59090909090909,131,0,0
3,/voti-e-risultati/1/risultati/giudizio-maggior...,24,16,1.5,17.625,53,0,0
4,/voti-e-risultati/2/vota/giudizio-maggioritario/,15,10,1.5,31.7,48,0,0


In [75]:
users_on_pages_classes.head(5)

,Titolo pagina e nome schermata,Visualizzazioni,Utenti,Visualizzazioni per utente,Durata media del coinvolgimento,Conteggio eventi,Conversioni,Entrate totali
0,Crea e Vota Sondaggi | Vota Meglio,339,52,6.519230769230769,85.76923076923077,991,0,0
1,Gestione sondaggi | Vota Meglio,59,21,2.8095238095238093,21.61904761904762,158,0,0
2,Voto Giudizio Maggioritario | Vota Meglio,58,26,2.230769230769231,43.69230769230769,193,0,0
3,Risultati Giudizio Maggioritario | Vota Meglio,30,18,1.6666666666666667,28.77777777777778,71,0,0
4,Conferma voto Giudizio Maggioritario | Vota Me...,18,14,1.2857142857142858,8.071428571428571,49,0,0


### Integrazione manuale di alcuni dati

Per evitare inutili complicazioni, si integrano manualmente alcuni dati (copiandoli dalla dashboard di GA)

In [76]:
total_n_users = 62

## KPI per la prospettiva del votante

### Metrica #01: % di utenti che hanno dato un voto con MJ

Motivo: Ci aiuta a capire quanti dei nostri utenti conoscono MJ da votanti

Formula: $\frac{\#\text{ utenti che hanno dato almeno 1 voto con MJ}}{\#\text{ utenti tot}}$

Note: 
- per ricavare il numero di utenti che hanno votato almeno 1 voto con MJ, guardiamo il nro di utenti osservati su tutte le pagine di nome "Conferma voto Giudizio Maggioritario | Vota Meglio" 

In [77]:
n_users_who_voted_MJ = int(users_on_pages_classes.loc[
    users_on_pages_classes["Titolo pagina e nome schermata"] == \
        "Conferma voto Giudizio Maggioritario | Vota Meglio", 
    "Utenti"
].astype(int).values[0])

print(f"# utenti che hanno dato almeno 1 voto con MJ:\t{n_users_who_voted_MJ}")
print(f"% utenti che hanno dato almeno 1 voto con MJ:\t" + str(
    int(n_users_who_voted_MJ/total_n_users*100)/100
    ))

# utenti che hanno dato almeno 1 voto con MJ:	14
% utenti che hanno dato almeno 1 voto con MJ:	0.22


### Metrica #02: tasso di completamento del voto con MJ

Motivo: ci permette di capire quanto il processo di voto con MJ viene portato a termine; si ricava confrontando il numero di voti ricevuti con il numero di utenti che ha visualizzato i vari sondaggi di tipo MJ.

Formula: $\frac{\#\text{ voti a sondaggi MJ}}{\#\text{ utenti sulle pagine di voto dei sondaggi MJ}}$

Note:
- per tracciare il numero di voti a sondaggi di tipo MJ sommiamo il numero di utenti sulle varie pagine di riepilogo voto (quelle aventi path "/voti-e-risultati/*/riepilogo-voto/giudizio-maggioritario/")
- gli utenti sulle pagine di voto dei sondaggi MJ sono la somma di tutti gli utenti su pagine con path "/voti-e-risultati/*/vota/giudizio-maggioritario/"


In [79]:
votes_MJ = users_on_pages.loc[
    users_on_pages["Percorso pagina e classe schermata"].str.match(
        "/voti-e-risultati/.*/riepilogo-voto/giudizio-maggioritario/"), 
    "Utenti"
].astype(int).sum()

users_who_started_MJ_vote = users_on_pages.loc[
    users_on_pages["Percorso pagina e classe schermata"].str.match(
        "/voti-e-risultati/.*/vota/giudizio-maggioritario/"), 
    "Utenti"
].astype(int).sum()


print(f"# voti a sondaggi MJ:\t{votes_MJ}") 
print(f"# utenti sulle pagine di voto dei sondaggi MJ:\t{users_who_started_MJ_vote}") 
print("Tasso completamento processo voto MJ:\t" + \
    str(int(votes_MJ/users_who_started_MJ_vote*100)/100))

# voti a sondaggi MJ:	18
# utenti sulle pagine di voto dei sondaggi MJ:	35
Tasso completamento processo voto MJ:	0.51


### Metrica #03: Quanti degli utenti che hanno votato sondaggi a scelta singola hanno dato anche un voto con il metodo MJ

Motivo: visto che abbiamo deciso di "suggerire" di votare alcuni sondaggi a scelta singola anche tramite il metodo MJ, si può vedere che percentuale di utenti coglie questo suggerimento. Questa potrebbe essere una buona misura di quanto il MJ 

Formula: $\frac{\#\text{ voti a sondaggi SO (*)}}{\#\text{ voti agli stessi sondaggi con MJ}}$

Note:
- (*) <-- Si contano solo i voti a sondaggi etichettati come "votabili anche con MJ"
- per recuperare i voti dei sondaggi SO votabili anche tramite MJ si potrebbe selezionare i sondaggi il cui id appare sia nel path di conferma del voto SO ("/voti-e-risultati/\*/riepilogo-voto/opzione-singola/") che nel path di converma del voto MJ ("/voti-e-risultati/\*/riepilogo-voto/giudizio-maggioritario/")
- questa metrica non è stata ancora calcolata per mancanza di dati


## KPI per la prospettiva del proponente

# Metrica #01: % sondaggi di tipo MJ tra quelli creati da utenti

Motivo: è interessante vedere quanto effettivamente i proponenti di sondaggi scelgano il metodo del MJ per i sondaggi che creano.

Formula: $\frac{\#\text{ sondaggi MJ creati}}{\#\text{ sondaggi creati}}$

Note:
- per il momento non abbiamo perdisposto un metodo semplice per contare i sondaggi creati dalle visualizzazioni 
- un'idea per stimare i sondaggi creati potrebbe essere:
  - contare tutti gli ID di sondaggio tramite i path di voto 
  - escludere gli ID dei sondaggi che sappiamo essere stati creati da noi
- i sondaggi ad opzione singola si propone anche MJ possono contare come sondaggi MJ

In [92]:
# polls I wanna exclude because I know they have 
# been created by us
our_polls_ids = ["1", "2", "3"]

n_mj_polls = 0
n_polls = 0

for path in users_on_pages.loc[
        users_on_pages["Percorso pagina e classe schermata"].str.match(
            "/voti-e-risultati/.*/vota/"), 
        "Percorso pagina e classe schermata"
    ].to_list():

    # check id is not in exclude list
    if path.split("/")[2] in our_polls_ids:
        continue

    n_polls += 1
    
    if path.split("/")[4] == "giudizio-maggioritario":
        n_mj_polls += 1

print(f"# sondaggi MJ creati:\t{n_mj_polls}")
print(f"# sondaggi creati:\t{n_polls}")
print(f"% sondaggi di tipo MJ:\t" + \
    str(int(n_mj_polls/n_polls*100)/100))

# sondaggi MJ creati:	1
# sondaggi creati:	2
% sondaggi di tipo MJ:	0.5


## Altre proposte di metriche

- numero di click verso link di approfondimento su come funziona il metodo del giudizio maggioritario (sia verso un'ipotetica pagina interna, sia verso link esterni)
- numero di sondaggi \[di tipo MJ\] in rapporto al numero di visite al form di creazione; questa misura potrebbe essere utile per capire qual è la percentuale 